In [1]:
import requests

url = "https://api.elections.kalshi.com/trade-api/v2/exchange/status"

response = requests.get(url)

print(response.text)

{"exchange_active":true,"exchange_estimated_resume_time":null,"trading_active":true}


In [2]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

def load_private_key_from_file(file_path):
    with open(file_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,  # or provide a password if your key is encrypted
            backend=default_backend()
        )
    return private_key

In [3]:
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.exceptions import InvalidSignature

def sign_pss_text(private_key: rsa.RSAPrivateKey, text: str) -> str:
    message = text.encode('utf-8')
    try:
        signature = private_key.sign(
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.DIGEST_LENGTH
            ),
            hashes.SHA256()
        )
        return base64.b64encode(signature).decode('utf-8')
    except InvalidSignature as e:
        raise ValueError("RSA sign PSS failed") from e

In [4]:
import requests
import datetime
from pathlib import Path


current_time = datetime.datetime.now()
timestamp = current_time.timestamp()
current_time_milliseconds = int(timestamp * 1000)
timestampt_str = str(current_time_milliseconds)

private_key = load_private_key_from_file('src/secrets/kalshi_private_key.pem')
api_key = Path('src/secrets/kalshi-key-2.key').read_text().strip()

method = "GET"
base_url = 'https://api.elections.kalshi.com'
path='/trade-api/v2/portfolio/balance'

# Strip query parameters from path before signing
path_without_query = path.split('?')[0]
msg_string = timestampt_str + method + path_without_query
sig = sign_pss_text(private_key, msg_string)

headers = {
    'KALSHI-ACCESS-KEY': api_key,
    'KALSHI-ACCESS-SIGNATURE': sig,
    'KALSHI-ACCESS-TIMESTAMP': timestampt_str
}

response = requests.get(base_url + path, headers=headers)

print(response.text)

{"balance":71,"portfolio_value":56,"updated_ts":1769793808}


In [5]:
import requests

def get_all_markets(series_ticker, endpoint):
    """Fetch all markets for a series, handling pagination"""
    all_markets = []
    cursor = None
    base_url = f"https://api.elections.kalshi.com/trade-api/v2/{endpoint}"

    while True:
        # Build URL with cursor if we have one
        url = f"{base_url}?series_ticker={series_ticker}&limit=100"
        if cursor:
            url += f"&cursor={cursor}"

        response = requests.get(url)
        data = response.json()

        # Add markets from this page
        all_markets.extend(data[endpoint])

        # Check if there are more pages
        cursor = data.get('cursor')
        if not cursor:
            break

        print(f"Fetched {len(data[endpoint])} {endpoint}, total: {len(all_markets)}")

    return all_markets

# Example usage
page = "events"
markets = get_all_markets("KXFEDCHAIRNOM-29", page)
print(f"Total {page} found: {len(markets)}")

Total events found: 0


In [6]:

url = "https://api.elections.kalshi.com/trade-api/v2/markets?limit=10&status=open"
b = requests.get(url)

In [7]:
import requests

# Get orderbook for a specific market
market_ticker = "KXFEDCHAIRNOM-29-KW"
url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{market_ticker}/orderbook"

response = requests.get(url)
orderbook_data = response.json()

In [8]:
orderbook_data

{'orderbook': {'no': [[1, 606682]],
  'no_dollars': [['0.0100', 606682]],
  'yes': [[1, 114952],
   [2, 102145],
   [4, 1000],
   [5, 1000],
   [8, 81],
   [9, 90],
   [11, 518],
   [12, 16],
   [15, 20],
   [16, 1000],
   [17, 1000],
   [20, 2],
   [28, 250],
   [29, 3834],
   [30, 100827],
   [31, 15000],
   [32, 100],
   [34, 16346],
   [50, 50362],
   [52, 300],
   [55, 25100],
   [56, 10000],
   [58, 167],
   [60, 200],
   [67, 3333],
   [70, 3916],
   [73, 105],
   [80, 100],
   [83, 10],
   [85, 250],
   [86, 100],
   [87, 600],
   [88, 10100],
   [89, 15388],
   [90, 1824],
   [91, 25001],
   [92, 81],
   [93, 18],
   [95, 3963],
   [96, 1615],
   [97, 4774],
   [98, 402074]],
  'yes_dollars': [['0.0100', 114952],
   ['0.0200', 102145],
   ['0.0400', 1000],
   ['0.0500', 1000],
   ['0.0800', 81],
   ['0.0900', 90],
   ['0.1100', 518],
   ['0.1200', 16],
   ['0.1500', 20],
   ['0.1600', 1000],
   ['0.1700', 1000],
   ['0.2000', 2],
   ['0.2800', 250],
   ['0.2900', 3834],
   ['0

In [9]:
import requests

url = "https://api.elections.kalshi.com/trade-api/v2/exchange/announcements"

response = requests.get(url)

print(response.text)

{"announcements":[]}


In [10]:
import requests

url = "https://api.elections.kalshi.com/trade-api/v2/series/fee_changes"

response = requests.get(url)

print(response.text)

{"series_fee_change_arr":[]}


In [11]:
import requests

url = "https://api.elections.kalshi.com/trade-api/v2/exchange/schedule"

response = requests.get(url)

print(response.text)

{"schedule":{"maintenance_windows":[],"standard_hours":[{"end_time":"2200-12-01T00:00:00Z","friday":[{"close_time":"00:00","open_time":"00:00"}],"monday":[{"close_time":"00:00","open_time":"00:00"}],"saturday":[{"close_time":"00:00","open_time":"00:00"}],"start_time":"2024-12-01T00:00:00Z","sunday":[{"close_time":"00:00","open_time":"00:00"}],"thursday":[{"close_time":"03:00","open_time":"00:00"},{"close_time":"00:00","open_time":"05:00"}],"tuesday":[{"close_time":"00:00","open_time":"00:00"}],"wednesday":[{"close_time":"00:00","open_time":"00:00"}]}]}}


In [13]:
# https://api.elections.kalshi.com/trade-api/v2/markets


In [14]:
import requests
from datetime import datetime, timezone

BASE = "https://api.elections.kalshi.com/trade-api/v2"


def fetch_all_open_markets(limit=1000):
    markets = []
    cursor = None

    while True:
        params = {"status": "open", "limit": limit}
        if cursor:
            params["cursor"] = cursor

        r = requests.get(f"{BASE}/markets", params=params, timeout=30)
        r.raise_for_status()
        data = r.json()

        markets.extend(data.get("markets", []))
        cursor = data.get("cursor")

        if not cursor:
            break

    return markets


def export_open_markets_json():
    raw_markets = fetch_all_open_markets()

    out = {
        "metadata": {
            "generated_at": datetime.now(timezone.utc).isoformat(),
            "market_count": len(raw_markets),
            "event_count": 0,
            "series_count": 0,
        },
        "series": {},
    }

    for m in raw_markets:
        series_id = m.get("series_id", "UNKNOWN_SERIES")
        event_id = m.get("event_id", "UNKNOWN_EVENT")

        series = out["series"].setdefault(series_id, {
            "series_id": series_id,
            "events": {}
        })

        events = series["events"]
        event = events.setdefault(event_id, {
            "event_id": event_id,
            "event_title": m.get("event_title"),
            "close_ts": m.get("close_ts"),
            "markets": []
        })

        event["markets"].append({
            "ticker": m.get("ticker"),
            "title": m.get("title"),
            "status": m.get("status"),
            "yes_ask": m.get("yes_ask"),
            "no_ask": m.get("no_ask"),
            "volume": m.get("volume"),
            "open_interest": m.get("open_interest"),
            "close_ts": m.get("close_ts"),
        })

    out["metadata"]["series_count"] = len(out["series"])
    out["metadata"]["event_count"] = sum(
        len(s["events"]) for s in out["series"].values()
    )

    return out


In [15]:
import json
import datetime as dt

now = dt.datetime.now().date()
data = export_open_markets_json()

with open(f"kalshi_open_markets_{now}.json", "w") as f:
    json.dump(data, f, indent=2)


In [ ]:
from src.data.db_export_open_markets import fetch_open_markets

def test_fetch_open_markets():
    markets = fetch_open_markets()
    assert isinstance(markets, list)
    assert len(markets) > 0
    assert all(isinstance(m, dict) for m in markets)

test_fetch_open_markets()

Fetched page 1, +1000 markets (total=1000)
Fetched page 2, +1000 markets (total=2000)
Fetched page 3, +1000 markets (total=3000)
Fetched page 4, +1000 markets (total=4000)
Fetched page 5, +1000 markets (total=5000)
Fetched page 6, +1000 markets (total=6000)
Fetched page 7, +1000 markets (total=7000)
